In [ ]:
# 작성자 : 나한재 (성장전략) 
# 제목 : 고객사 별 월 별 마감 B2C 송장 건 수 - 고객사 (seller.poomgo.com) 데이터 요청
# 링크 : https://dohandsworkspace.slack.com/archives/C01MP08HSH2/p1711954418357869

In [1]:
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine, text
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
#pd.set_option('display.max_rows', None)

In [5]:
class Query:
    def querytemp(self):
        query = '''
        with b2c_invoice_cnt as (select --boxresource
        i."closeAt" at time zone 'Asia/Seoul' AS "마감일시",
        i.code AS "송장번호",
        lc.name AS "센터명",
        lc.code as c_code,
        p.name AS "고객사명",
        b.name AS "브랜드명",
        br."code" AS "박스바코드",
        br."name" AS 박스명,
        (CASE
        WHEN (i."deliverySheetId" is not null) THEN 'Y'
        ELSE 'N'
        END) AS "B2B출고여부"
        FROM
        logistics.invoices i
        LEFT JOIN commerce. "orderSheets" os ON os.id = i. "orderSheetId"
        left join wms.operations op on op."invoiceId" = i.id
        LEFT JOIN master.partners p ON p.id = i. "partnerId"
        LEFT JOIN commerce.channels c ON c.id = i. "channelId"
        LEFT JOIN logistics. "logisticsCenters" lc ON lc.id = i. "logisticsCenterId"
        LEFT JOIN logistics. "logisticsPartners" lp ON lp.id = i. "logisticsPartnerId"
        LEFT OUTER JOIN "master"."users" AS "closeUser" ON "i"."closeUserId" = "closeUser"."id"
        LEFT OUTER JOIN "master"."codes" AS "cargoStatusCode" ON "i"."cargoStatusCodeKey" = "cargoStatusCode"."key"
        LEFT JOIN master."partnerResources" AS pr ON pr."resourceId" = op."resourceId" AND pr."partnerId" = op."partnerId"
        LEFT JOIN master."brands" AS b ON b."id" = pr."brandId"
        LEFT JOIN master."resources" AS br ON br."id" = i."boxResourceId"		
        WHERE
        i. "tempOrderSheetId" IS null
        AND "i". "isConfirmed" = TRUE
        AND "i". "isClosed" = TRUE
        AND "i". "isCanceled" = FALSE
        AND "i". "isPostponed" = FALSE
        AND "i". "isAccepted" = TRUE
        AND i."closeAt" >= '2024-03-01 00:00:00+09' --일정 수정
        AND i."closeAt" < '2024-04-01 00:00:00+09' --일정 수정
        AND lc."code" in ('ES1', 'ES2' , 'ES3', '', '', '')
        order by i."closeAt" asc) select * from b2c_invoice_cnt;
        '''
        return query

In [6]:
class ConnectDB:
    def __init__(self):
        self.username = 'wy.jeon' #str형 / 명칭
        self.password = '1234' #str형
        self.ip_address = 'poomgo-production-cluster.cluster-cfsilzqil2ni.ap-northeast-2.rds.amazonaws.com' #str형 / 주소 000.000.000.000
        self.port_number = 5432 #int형
        
    def engine_backend(self): #center name
        db_name = 'poomgo_backend' #str형 / 명칭
        try :
            engine = create_engine(f"postgresql+psycopg2://{self.username}:{self.password}@{self.ip_address}:{self.port_number}/{db_name}")
        except Exception as e:
            return print('Exception : ', e)
        else:
            print('completed to create engine')
            return engine

    def engine_now(self): #cs , return information
        db_name = 'poomgo_now' #str형 / 명칭
        try :
            engine = create_engine(f"postgresql+psycopg2://{self.username}:{self.password}@{self.ip_address}:{self.port_number}/{db_name}")
        except Exception as e:
            return print('Exception : ', e)
        else:
            return engine

    def read_data(self, query, engine):
        #connect = engine
        data = pd.read_sql(query, engine)
        return data

In [7]:
if __name__ == '__main__':
    q = Query()
    querytemp = q.querytemp()
    db = ConnectDB()
    engine_temp = db.engine_backend() 
    df_temp = db.read_data(querytemp, engine_temp)
    print('데이터 추출 완료!')
    print('End Of Script')

completed to create engine
데이터 추출 완료!
End Of Script


In [8]:
df_temp

,마감일시,송장번호,센터명,고객사명,브랜드명,박스바코드,박스명,B2B출고여부
0,2024-03-01 10:09:15.941,532729597304,파주 센터,코지모해,None,2023122800001,자체박스1호(2개입),N
1,2024-03-01 10:09:56.102,532729597396,파주 센터,코지모해,None,8829012016128,A1(극소형),N
2,2024-03-01 10:10:10.711,532729597466,파주 센터,코지모해,None,8829012016128,A1(극소형),N
3,2024-03-01 10:11:27.823,532729597352,파주 센터,니드코,니드코,8829012822187,B1(극소형),N
4,2024-03-01 10:12:00.846,532729597293,파주 센터,니드코,니드코,8829012822187,B1(극소형),N
...,...,...,...,...,...,...,...,...
1435218,2024-03-31 23:48:54.603,532850203712,음성 2센터,클럽클리오,페리페라,8829012016128,A1(극소형),N
1435219,2024-03-31 23:48:54.603,532850203712,음성 2센터,클럽클리오,페리페라,8829012016128,A1(극소형),N
1435220,2024-03-31 23:49:48.228,532850203200,음성 2센터,크리오,크리오,8829012420147,A2(극소형),N
1435221,2024-03-31 23:49:48.228,532850203200,음성 2센터,크리오,크리오,8829012420147,A2(극소형),N


In [12]:
test = df_temp[(df_temp['센터명']=='음성 1센터')&(df_temp['고객사명']=='다담다')]

In [13]:
test.head()

,마감일시,송장번호,센터명,고객사명,브랜드명,박스바코드,박스명,B2B출고여부
25677,2024-03-01 18:35:06.386,532729506470,음성 1센터,다담다,바로사인,01,바로사인 박스 (1호),N
26025,2024-03-01 18:42:19.383,532729576282,음성 1센터,다담다,바로사인,8829013628245,C1(소형),N
26026,2024-03-01 18:42:19.383,532729576282,음성 1센터,다담다,바로사인,8829013628245,C1(소형),N
26061,2024-03-01 18:43:06.040,532729870853,음성 1센터,다담다,바로사인,8829013224201,B2(극소형),N
26062,2024-03-01 18:43:06.040,532729870853,음성 1센터,다담다,바로사인,8829013224201,B2(극소형),N


In [14]:
len(test['송장번호'].unique())

5648

In [15]:
df_temp.to_csv('./[0401]_3월분_고객사별_마감B2C_송장건수.csv')